In [1]:
!pip install numpy
!pip install matplotlib
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   -------------------- ------------------- 6.6/12.8 MB 34.9 MB/s eta 0:00:01
   ---------------------------- ----------- 9.2/12.8 MB 22.3 MB/s eta 0:00:01
   ---------------------------------------- 12.8/12.8 MB 23.5 MB/s  0:00:00
Defaulting to user installation because normal site-packages is not writeable
  Using cached contourpy-1.3.3-cp313-cp313-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.9-cp313-cp313-win_amd64.whl.metadata (6.4 kB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   --------- ------------------------------ 1.8/8.1 MB 9.9 MB/s eta 0:00:01
   ------------------------ --------------- 5.0/8.1 MB 12.7 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 14.4 MB/s  0:00:00
Using cached contourpy-1.3.3-cp

In [2]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

In [ ]:
import json

# Defining the 8 u120 instances
problems = [
  
  {
    "name": "u120_00",
    "capacity": 150,
    "best": 48,
    "items": [42,69,67,57,93,90,38,36,45,42,33,79,27,57,44,84,86,92,46,38,85,33,82,73,49,70,59,23,57,72,74,69,33,42,28,46,30,64,29,74,41,49,55,98,80,32,25,38,82,30,35,39,57,84,62,50,55,27,30,36,20,78,47,26,45,41,58,98,91,96,73,84,37,93,91,43,73,85,81,79,71,80,76,83,41,78,70,23,42,87,43,84,60,55,49,78,73,62,36,44,94,69,32,96,70,84,58,78,25,80,58,66,83,24,98,60,42,43,43,39]
  },
  {
    "name": "u120_01",
    "capacity": 150,
    "best": 49,
    "items": [97,57,81,62,75,81,23,43,50,38,60,58,70,88,36,90,37,45,45,39,44,53,70,24,82,81,47,97,35,65,74,68,49,55,52,94,95,29,99,20,22,25,49,46,98,59,98,60,23,72,33,98,80,95,78,57,67,53,47,53,36,38,92,30,80,32,97,39,80,72,55,41,60,67,53,65,95,20,66,78,98,47,100,85,53,53,67,27,22,61,43,52,76,64,61,29,30,46,79,66,27,79,98,90,22,75,57,67,36,70,99,48,43,45,71,100,88,48,27,39]
  },
  {
    "name": "u120_02",
    "capacity": 150,
    "best": 46,
    "items": [38,100,60,42,20,69,24,23,92,32,84,36,65,84,34,68,64,33,69,27,47,21,85,88,59,61,50,53,37,75,64,84,74,57,83,28,31,97,61,36,46,37,96,80,53,51,68,90,64,81,66,67,80,37,92,67,64,31,94,45,80,28,76,29,64,38,48,40,29,44,81,35,51,48,67,24,46,38,76,22,30,67,45,41,29,41,79,21,25,90,62,34,73,50,79,66,59,42,90,79,70,66,80,35,62,98,97,37,32,75,91,91,48,26,23,32,100,46,29,26]
  },
  {
    "name": "u120_03",
    "capacity": 150,
    "best": 49,
    "items": [29,26,83,82,92,95,87,63,57,100,63,65,81,46,42,95,90,80,53,27,84,40,22,97,20,73,63,95,46,42,47,40,26,88,49,24,92,87,68,95,34,82,84,43,54,73,66,32,62,48,99,90,86,28,25,25,89,67,96,35,33,70,40,59,32,94,34,86,35,45,25,76,80,42,91,44,91,97,60,29,45,37,61,54,78,56,74,74,45,21,96,37,75,100,58,84,85,56,54,71,52,79,43,35,27,70,31,47,35,26,30,97,90,80,58,60,73,46,71,39]
  },
  {
    "name": "u120_04",
    "capacity": 150,
    "best": 50,
    "items": [42,98,27,21,71,71,78,76,57,24,91,84,35,25,77,96,97,89,30,86,81,39,75,66,85,36,60,56,50,75,75,37,87,95,21,99,42,57,31,37,42,40,69,91,45,97,84,90,52,43,68,53,37,65,79,73,92,87,20,20,73,42,52,20,24,76,71,72,21,21,82,92,78,87,50,41,31,73,89,59,88,40,71,69,45,57,49,68,84,32,69,77,92,98,57,39,32,23,99,91,48,21,70,43,73,69,65,57,67,28,84,42,61,92,82,34,74,55,60,69]
  },
  {
    "name": "u120_05",
    "capacity": 150,
    "best": 48,
    "items": [26,25,67,77,67,79,47,84,50,21,87,83,44,88,78,53,78,37,47,52,32,88,85,82,55,41,60,66,78,72,34,64,20,60,100,62,80,34,68,38,32,32,37,82,98,90,58,97,56,34,70,39,56,69,36,20,99,84,53,27,88,53,42,45,42,31,54,60,55,27,36,31,39,91,45,97,26,80,41,56,70,97,48,87,23,32,75,100,97,51,78,78,21,72,72,79,46,30,48,27,95,48,67,58,46,92,21,82,91,40,56,24,94,44,91,92,81,24,84,44]
  },
  {
    "name": "u120_06",
    "capacity": 150,
    "best": 48,
    "items": [83,37,98,85,88,95,29,35,100,55,48,27,20,66,62,52,88,59,97,91,81,81,86,48,43,60,72,88,90,48,38,60,53,55,90,48,55,57,59,25,51,22,43,31,52,89,96,58,63,27,46,43,30,44,71,66,64,28,83,88,42,92,95,36,24,62,44,82,59,31,96,44,61,78,72,62,76,65,22,41,27,85,80,72,100,29,27,43,83,32,33,53,95,99,20,23,72,50,50,27,89,53,75,81,34,27,69,48,84,37,69,54,51,49,49,54,100,55,45,83]
  },
  {
    "name": "u120_07",
    "capacity": 150,
    "best": 49,
    "items": [61,96,91,37,53,76,50,66,70,87,92,35,53,95,47,56,55,86,32,99,83,88,41,63,77,60,66,53,79,81,96,34,99,47,74,87,44,77,52,99,69,64,94,38,69,61,98,40,84,89,49,64,53,41,34,85,35,55,61,68,100,75,98,36,44,57,24,60,45,48,60,94,71,70,64,62,93,20,69,37,63,61,26,54,89,46,54,50,32,71,62,40,26,59,62,27,60,50,74,34,40,70,56,23,66,57,43,45,65,25,82,82,37,66,47,44,94,23,24,51]
  }
]

with open("binpack_u120.json", "w") as f:
    json.dump(problems, f, indent=2)




In [ ]:

class AntColony:
   def __init__(self, sizes, capacity, n_ants, n_best, n_iterations, decay, alpha, beta, R_max, temperature, shuffle_prob):

    self.sizes = sizes
    self.capacity = capacity
    self.n = len(sizes)

    self.n_ants = n_ants
    self.n_best = n_best
    self.n_iterations = n_iterations
    self.decay = decay
    self.alpha = alpha
    self.beta = beta
    self.R_max = R_max

    # Pheromone per rank 
    self.pheromone = np.ones(R_max + 1)

    # Temperatur for softmax exploration
    self.temperature = temperature

    self.shuffle_prob = shuffle_prob

    # --- Helper functions ---

   def heuristic(self, residual):
      # Chooses the bin with the smallest leftover space
      return 1.0 / (1.0 + residual)

   def choice_probabilities(self, feasible_bins, new_residual):
      # Compute seleciton probability for each bin (existing bin + new bin)
      options = []

      #Existing bins
      for rank, residual in feasible_bins:
        limited_rank = min(rank, self.R_max)
        pheromone_strength = self.pheromone[limited_rank] ** self.alpha
        heuristic_value = self.heuristic(residual) ** self.beta
        options.append((limited_rank, pheromone_strength * heuristic_value))

      #New bin
      pheromone_new = self.pheromone[0] ** self.alpha
      heuristic_new = self.heuristic(new_residual) ** self.beta
      options.append((0, pheromone_new * heuristic_new))

      # convert to numpy array for temperatur scaling
      ranks, scores = zip(*options)
      scores = np.array(scores)

      probs = scores ** (1 / self.temperature)
      probs /= probs.sum()
      return list(zip(ranks, probs))


   def sample_rank(self, probabilities):
      # Randomly choose a rank based on its probability
      random_value = np.random.random()
      running_total = 0.0
      for rank, prob in probabilities:
        running_total += prob
        if random_value <= running_total:
          return rank
      return probabilities[-1][0]

   # --- Construct one solution ---

   def construct_solution(self):
      bins = [] # each bin holds item indices
      loads = [] # total load of each bin
      ranks_used = [] # track ranks chosen for later phermonoes update

      order=list(range(self.n))
      if np.random.random() < self.shuffle_prob:
        np.random.shuffle(order)
      else:
        order.sort(key=lambda i: -self.sizes[i])

      for item in order:
        item_size = self.sizes[item]

        # find bins where the item fits
        feasible = []
        for bin_index in range(len(loads)):
          if loads[bin_index] + item_size <= self.capacity:
            residual_after = self.capacity - (loads[bin_index] + item_size)
            feasible.append((bin_index, residual_after))
        feasible_ranked = [(i + 1, res) for i, (_, res) in enumerate(sorted(feasible,  key=lambda x: x[1]))]

        # calculate probabilities
        new_residual = self.capacity - item_size
        probabilities = self.choice_probabilities(feasible_ranked, new_residual)
        chosen_rank = self.sample_rank(probabilities)
        ranks_used.append(chosen_rank)

        # place the item
        if chosen_rank == 0:
           bins.append([item])
           loads.append(item_size)
        else:
          index = min(chosen_rank - 1, len(feasible) - 1)
          bin_index = feasible[index][0]
          bins[bin_index].append(item)
          loads[bin_index] += item_size

      num_bins = len(bins)
      total_unused = num_bins * self.capacity - sum(loads)
      return bins, loads, ranks_used, num_bins, total_unused

   def run(self):
    best_solution = None
    best_bins = float("inf")
    best_progress = []

    for iteration in range(self.n_iterations):
      all_solutions = []
      for _ in range(self.n_ants):
        bins, loads, ranks_used, num_bins, total_unused = self.construct_solution()
        all_solutions.append((bins, loads, ranks_used, num_bins, total_unused))

      # Sort by number of bins used and unused space
      all_solutions.sort(key=lambda x: (x[3], x[4]))

      # Evaporate pheromone
      self.pheromone *= self.decay

      # Update pheromone for the n_best solutions
      for i in range(self.n_best):
        bins, loads, ranks_used, num_bins, total_unused = all_solutions[i]
        for r in ranks_used:
          self.pheromone[r] += 1.0 / num_bins

      # track the best
      best_iteration = all_solutions[0]
      if best_iteration[3] < best_bins:
        best_bins = best_iteration[3]
        best_solution = best_iteration
      best_progress.append(best_bins)

      print(f"Iteration {iteration+1}/{self.n_iterations} -> Best bins: {best_bins}")

    print("\nBest overall solution used", best_bins, "bins")
    bins, loads, _, _, _ = best_solution
    for i, (b, load) in enumerate(zip(bins, loads)):
      print(f" Bin {i}: load={load}, items={[self.sizes[j] for j in b]}")

    return  best_solution, best_progress

# --- Load JSON dataset ---

with open("binpack_u120.json", "r") as f:
    problems = json.load(f)

results = []

for prob in problems:
    print(f"\n Running instance: {prob['name']}")
    sizes = prob["items"]
    capacity = prob["capacity"]
    best_known = prob["best"]

    start_time = time.time()

    ant_colony = AntColony(
        sizes, capacity,
        n_ants=50,
        n_best=5,
        n_iterations=200,
        decay=1,
        alpha=3,
        beta=1,
        R_max=10,
        temperature=2.0,
        shuffle_prob=0.3
    )

    best_solution, progress = ant_colony.run()
    bins, loads, ranks_used, num_bins, total_unused = best_solution

    end_time = time.time()
    runtime = round(end_time - start_time, 3)

    plt.figure()
    plt.plot(range(1, len(progress) + 1), progress, marker='o')
    plt.xlabel("Iteration")
    plt.ylabel("Best bins")
    plt.title(f"Convergence - {prob['name']}")
    plt.grid(True)
    plt.show()

    plt.figure()
    plt.bar(range(len(loads)), loads, color='skyblue')
    plt.axhline(capacity, color='red', linestyle='--', label='Capacity')
    plt.xlabel("Bin index")
    plt.ylabel("Load")
    plt.title(f"Load Distribution - {prob['name']}")
    plt.legend()
    plt.show()

    # Save results for summary table
    results.append({
        "Instance": prob["name"],
        "Best Known": best_known,
        "ACO Bins": num_bins,
        "Unused Space": total_unused,
        "Runtime (s)": runtime
    })

# --- Display summary table ---

df = pd.DataFrame(results)
print("\n=== Summary Table ===")
print(df)

# Plot table using matplotlib
fig, ax = plt.subplots(figsize=(10, 4))
ax.axis('tight')
ax.axis('off')
table_data = df.values
column_labels = df.columns
ax.table(cellText=table_data, colLabels=column_labels, loc='center')
plt.title("ACO Bin Packing Results - u120 Instances")
plt.show()


# Optimize Classic Benchmark Functions using Particle Swarm Optimization (PSO)

In [ ]:
#--- Import Dependencies ---#
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
#--- Sphere function that we are attempting to optimiz (minimize) ---#
def sphere(x):
    return np.sum(np.square(x))


#--- Rosenbrock function that we are attempting to optimize (minimize) ---#
def rosenbrock(x):
    total = 0
    for i in range(len(x) - 1):
        part1 = 100*(x[i+1] - x[i]**2)**2
        part2 = (1-x[i])**2
        total += (part1 + part2)

    return total


#--- Rastrigin function that we are attempting to optimize (minimize) ---#
def rastrigin(x):
    total = 10 * len(x) + np.sum(np.square(x) - 10 * np.cos(2 * np.pi * x))
    return total

    for i in range(len(x)):
        part1 = x[i]**2
        part2 = 10*np.cos(2*np.pi*x[i])
        total += (part1-part2)



#--- Ackley function that we are attempting to optimize (minimize) ---#
def ackley(x):
    n = len(x)
    total1 = -20*np.exp(-0.2*np.sqrt(1/n*np.sum(np.square(x))))
    total2 = -np.exp(1/n * np.sum(np.cos(2*np.pi*x)))
    return total1 + total2 + 20 + np.exp(1)

In [ ]:
#--- Main ---#
class Particle:
    def __init__(self, bounds):
        self.num_dimensions = len(bounds)
        self.bounds = bounds
        self.position_i = np.array([np.random.uniform(b[0], b[1]) for b in bounds])  # Current position
        range_size = [b[1] - b[0] for b in bounds]
        self.velocity_i = np.array([np.random.uniform(-0.2*r, 0.2*r) for r in range_size])  # Current velocity
        self.pos_best_i = np.copy(self.position_i)  # Best known position of this particle
        self.err_best_i = np.inf  # Best known error of this particle
        self.err_i = np.inf       # Current err


    def evaluate (self, costFunc):
        self.err_i = costFunc(self.position_i)

        if self.err_i < self.err_best_i:
            self.pos_best_i = np.copy(self.position_i)
            self.err_best_i = self.err_i


    def update_velocity(self, pos_best_g):
        inertia_weight = 0.7
        cognitive_coef, social_coef = 1.5, 1.5
        r1 = np.random.random(self.num_dimensions)
        r2 = np.random.random(self.num_dimensions)

        vel_cognitive = cognitive_coef * r1 * (self.pos_best_i - self.position_i)
        vel_social = social_coef * r2 * (pos_best_g - self.position_i)
        self.velocity_i = inertia_weight * self.velocity_i + vel_cognitive + vel_social
        v_max = np.array([0.5 * (b[1] - b[0]) for b in self.bounds])
        self.velocity_i = np.clip(self.velocity_i, -v_max, v_max)


    def update_position(self, bounds):
        for i in range(0, self.num_dimensions):
            self.position_i[i] = self.position_i[i] + self.velocity_i[i]

            if self.position_i[i] > bounds[i][1]:
                self.position_i[i] = bounds[i][1]

            if self.position_i[i] < bounds[i][0]:
                self.position_i[i] = bounds[i][0]

In [ ]:
#--- PSO Loop ---#

class PSO():
    def __init__(self, costFunc, bounds, num_particles=30, max_iterations=300):
        self.costFunc = costFunc
        self.bounds = bounds
        self.num_particles = num_particles
        self.max_iterations = max_iterations
        self.err_best_g = np.inf
        self.pos_best_g = None

    def run(self):
        swarm = [Particle(self.bounds) for _ in range(self.num_particles)]
        best_fitness_curve = []
        for iteration in range(self.max_iterations):
            for particle in swarm:
                particle.evaluate(self.costFunc)
                if particle.err_i < self.err_best_g:
                    self.pos_best_g = np.copy(particle.position_i)
                    self.err_best_g = particle.err_i
            best_fitness_curve.append(self.err_best_g)

            #--- Early stopping condition ---#
            func_name = self.costFunc.__name__.lower()
            if func_name in ['sphere', 'rosenbrock'] and self.err_best_g < 1e-8:
                #print(f"Stopping early at iteration {iteration} (Sphere/Rosenbrock)")
                break

            if func_name in ['rastrigin', 'ackley'] and self.err_best_g < 1e-4:
                #print(f"Stopping early at iteration {iteration} (Rastrigin/Ackley)")
                break

            #if iteration == self.max_iterations - 1:
            #print("Maximum number of iterations reached")

            for particle in swarm:
                particle.update_velocity(self.pos_best_g)
                particle.update_position(self.bounds)

            #print("Best position:", self.pos_best_g)
            #print("Best fitness:", self.err_best_g)
        evaluations_used = len(best_fitness_curve) * self.num_particles
        return self.pos_best_g, self.err_best_g, best_fitness_curve, evaluations_used



In [ ]:
def plot_single_convergence(results, function_name):
    curves = results[function_name]["curves"]
    plt.figure(figsize=(6,4))
    plt.plot(curves[0])  # first run
    plt.title(f"Convergence curve – {function_name}")
    plt.xlabel("Iteration")
    plt.ylabel("Best fitness (gbest)")
    plt.yscale("log")  # optional for clarity
    plt.grid(True)
    plt.show()

def plot_mean_convergence(results, function_name):
    curves = results[function_name]["curves"]
    max_len = max(len(c) for c in curves)
    # Pad shorter runs
    padded = np.array([np.pad(c, (0, max_len - len(c)), 'edge') for c in curves])
    mean_curve = np.mean(padded, axis=0)
    std_curve = np.std(padded, axis=0)

    plt.figure(figsize=(6,4))
    plt.plot(mean_curve, label="Mean gbest")
    plt.fill_between(range(max_len),
                     mean_curve - std_curve,
                     mean_curve + std_curve,
                     color="gray", alpha=0.3)
    plt.title(f"Mean convergence curve – {function_name}")
    plt.xlabel("Iteration")
    plt.ylabel("Best fitness (gbest)")
    plt.yscale("log")
    plt.legend()
    plt.grid(True)
    plt.show()


def compare_functions(results, functions):
    plt.figure(figsize=(7,5))
    for f in functions:
        curves = results[f]["curves"]
        max_len = max(len(c) for c in curves)
        padded = np.array([np.pad(c, (0, max_len - len(c)), 'edge') for c in curves])
        plt.plot(np.mean(padded, axis=0), label=f)
    plt.title("PSO Convergence Comparison")
    plt.xlabel("Iteration")
    plt.ylabel("Mean best fitness (log scale)")
    plt.yscale("log")
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_boxplots(results, dimensions):
    plt.figure(figsize=(7,5))
    data = [ [r for curve_set in results[f]["curves"] for r in [curve_set[-1]]]  # final fitness per run
             for f in results.keys() ]
    labels = list(results.keys())
    plt.boxplot(data, labels=labels, showmeans=True)
    plt.yscale("log")  # useful since fitness values vary greatly
    plt.title(f"Final Fitness Across 30 Runs for {dimensions} Dimensions")
    plt.ylabel("Final Best Fitness (log scale)")
    plt.grid(True)
    plt.show()

In [ ]:
def run_experimental(num_runs=30, num_dimensions=2):
    results = {}

    functions = [sphere, rosenbrock, rastrigin, ackley]

    for costFunc in functions:
        print(f"\nFunction: {costFunc.__name__}, Dimensions: {num_dimensions}, Runs: {num_runs}")
        if costFunc == rosenbrock:
            bounds = np.array([(-5, 10) for _ in range(num_dimensions)])
        elif costFunc == rastrigin:
            bounds = np.array([(-5.12, 5.12) for _ in range(num_dimensions)])
        elif costFunc == ackley:
            bounds = np.array([(-32.768, 32.768) for _ in range(num_dimensions)])
        else:
            bounds = np.array([(-5.12, 5.12) for _ in range(num_dimensions)])

        fitnesses, all_curves, evaluations = [], [], []
        start_time = time.time()

        for _ in range(num_runs):
            n_particles = 50 if num_dimensions == 30 else 30
            pso = PSO(costFunc=costFunc, bounds=bounds, num_particles=n_particles)
            best_pos, best_fit, best_curve, evals = pso.run()
            fitnesses.append(best_fit)
            evaluations.append(evals)
            all_curves.append(best_curve)


        runtime = time.time() - start_time
        success_threshold = 1e-7 if costFunc in [sphere, rosenbrock] else 1e-3
        success_rate = np.mean(np.array(fitnesses) < success_threshold)

        results[costFunc.__name__] = {
            "mean": np.mean(fitnesses),
            "median": np.median(fitnesses),
            "std": np.std(fitnesses),
            "best": np.min(fitnesses),
            "worst": np.max(fitnesses),
            "success_rate": success_rate,
            "mean_evaluations": np.mean(evaluations),
            "runtime_sec": runtime,
            "curves": all_curves
        }

        print(f"→ mean: {np.mean(fitnesses):.3e}, std: {np.std(fitnesses):.3e}, "
              f"best: {np.min(fitnesses):.3e}, worst: {np.max(fitnesses):.3e}, "
              f"success_rate: {success_rate*100:.2f}, time: {runtime:.2f}s")

        #plot_mean_convergence(results, costFunc.__name__)
        if costFunc == ackley:
            print("\n\n")

    return results

In [ ]:
np.random.seed(42)
for num_dimensions in [2, 10, 30]:
  results = run_experimental(num_dimensions=num_dimensions)
  plot_boxplots(results, num_dimensions)
  print("\n\n")